In [1]:
import pandas as pd
import numpy as np
import re

pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

DATA_PATH = '/Users/eugenganenco/Desktop/srealtyAnalysis/data/DataFile 16_11_2022_22_43/housesDf_17_11_2022_00_06_15.csv translatedWithCoordAndPrice.csv'
df = pd.read_csv(DATA_PATH)
df = df.loc[df['price'] != 0]
#df = df.loc[df['price'] < 10000000]
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [2]:
def adaptToSQL(dfColumns):
    return {x: x.lower().replace(" ", "_").replace("-", "_").replace(r"/", "_") \
                      .replace("\\", "_").replace(".", "_").replace("$", "").replace("%", "") for x in dfColumns}
df.rename(columns=adaptToSQL(df.columns), inplace = True)

In [3]:
print(df['house_type'].unique())
print(df['housetype'].unique())
print(df['district'].unique())

['Patrový' 'Přízemní']
['rodinne-domy\n' 'vily\n' 'chalupy\n' 'chaty\n' 'projekty-na-klic\n'
 'zemedelske-usedlosti\n' 'pamatky-jine\n' 'vicegeneracni-domy\n']
['cheb' 'karlovy-vary' 'sokolov' 'tachov' 'rokycany' 'plzen-sever' 'plzen'
 'plzen-jih' 'klatovy' 'domazlice' 'usti-nad-labem' 'teplice' 'most'
 'louny' 'litomerice' 'decin' 'chomutov' 'benesov' 'beroun' 'kladno'
 'kolin' 'kutna-hora' 'melnik' 'mlada-boleslav' 'nymburk' 'praha-vychod'
 'praha-zapad' 'pribram' 'rakovnik' 'praha-1' 'praha-2' 'praha-3'
 'praha-4' 'praha-5' 'praha-6' 'praha-7' 'praha-8' 'praha-9' 'praha-10'
 'ceske-budejovice' 'cesky-krumlov' 'jindrichuv-hradec' 'pisek'
 'prachatice' 'strakonice' 'tabor' 'havlickuv-brod' 'jihlava' 'pelhrimov'
 'trebic' 'zdar-nad-sazavou' 'chrudim' 'pardubice' 'svitavy'
 'usti-nad-orlici' 'hradec-kralove' 'jicin' 'nachod' 'rychnov-nad-kneznou'
 'trutnov' 'ceska-lipa' 'jablonec-nad-nisou' 'liberec' 'semily' 'blansko'
 'breclav' 'brno' 'brno-venkov' 'hodonin' 'vyskov' 'znojmo' 'olomouc

In [4]:
df.head(1)

,link,location,district,housetype,total_price,note_on_price,job_id,update,building,object_status,location_of_the_house,object_location,house_type,the_floor,usable_area,land_area,parking,move_in_date,year_of_reconstruction,water,heating,garbage,electricity,transport,energy_efficiency_of_the_building,equipment,lift,companyname,proximityindex,index,built_up_area,store,garage,gas,telecommunication,discounted,original_price,garden_area,year_of_approval,swimming_pool,cena,floor_area,communication,indicator_of_energy_efficiency_of_the_building,certificate_of_energy_performance_of_the_building,barrier_free,ceiling_height,id,housing_costs,number_of_flats,starting_price,expert_opinion,minimum_bid,auction_principal,type_of_auction,place_of_auction,the_date_of_the_auction,the_date_of_the_1st_inspection,the_date_of_the_2nd_inspection,auction_decree,expert_opinion_1,pool_area,condition,date_of_completion_of_construction,soil_construction,sale_start_date,inspection_date,inspection_date_to,minimum_purchase_price,share_size,number_of_owners,locationlat,locationlong,price
0,https://www.sreality.cz/detail/prodej/dum/rodinny/skalna-skalna-chebska/4024649548,"Chebská, Skalná",cheb,rodinne-domy\n,6 900 000 Kč za nemovitost,V případě více zájemců může RK využít pro výběr kupujícího formu aukce.,817946,Včera,Cihlová,Dobrý,Samostatný,Okraj obce,Patrový,2,196 m2,681 m2,3.0,Ihned,2022.0,Dálkový vodovod,Lokální plynové,"Veřejná kanalizace, Jímka",230V,"Silnice, Autobus",Třída C - Úsporná č. 148/2007 Sb. podle vyhlášky,0,0.0,M&M reality,846.615385,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.1719781,12.3687621,6900000


In [5]:
## First lets handle categorical  which are missing
## Move to data analysis file

def percentageMissing(feature, data):
    missingValues = data[feature].apply(lambda x: 1 if pd.isna(x) else 0).sum()
    return round((missingValues/len(data.axes[0])) * 100, 3)

def getMissingDict(WithNan, data):
    missingDict = {}
    for feature in WithNan:
        percentage = percentageMissing(feature, data)
        missingDict[feature] = percentage
    return sorted(missingDict.items(), key=lambda x: x[1], reverse=True)

WithNan=[feature for feature in df.columns if df[feature].isnull().sum() > 0]
missingDict = getMissingDict(WithNan, df)

for index, feature in enumerate(missingDict):
    print('Variable "{}" has {}% of its values missing'.format(missingDict[index][0], missingDict[index][1]))
    

Variable "cena" has 100.0% of its values missing
Variable "starting_price" has 100.0% of its values missing
Variable "expert_opinion" has 100.0% of its values missing
Variable "the_date_of_the_1st_inspection" has 100.0% of its values missing
Variable "the_date_of_the_2nd_inspection" has 100.0% of its values missing
Variable "minimum_purchase_price" has 100.0% of its values missing
Variable "minimum_bid" has 99.993% of its values missing
Variable "auction_principal" has 99.993% of its values missing
Variable "type_of_auction" has 99.993% of its values missing
Variable "place_of_auction" has 99.993% of its values missing
Variable "the_date_of_the_auction" has 99.993% of its values missing
Variable "auction_decree" has 99.993% of its values missing
Variable "expert_opinion_1" has 99.993% of its values missing
Variable "number_of_owners" has 99.986% of its values missing
Variable "share_size" has 99.971% of its values missing
Variable "inspection_date_to" has 99.899% of its values missing


## Split data

In [6]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size=0.2, random_state=17)

In [7]:
X_train = train_set.drop('price', axis=1)
Y_train = train_set['price'].copy()

## Adding data about regional characterisitcs

In [8]:
districts_data = pd.read_csv('/Users/eugenganenco/Desktop/srealtyAnalysis/data/Regional Data/Districts_of_the_Czech_Republic_1.csv')
regions_data = pd.read_csv('/Users/eugenganenco/Desktop/srealtyAnalysis/data/Regional Data/Regions_of_the_Czech_Republic_1.csv')

In [9]:
districts_data

,Region,District,Area\n(km2),Population\n(2022),Mun.
0,Prague,Prague,496,"1,275,406",1
1,Central Bohemian Region,Benešov,"1,475","99,323",114
2,Central Bohemian Region,Beroun,662,"96,624",85
3,Central Bohemian Region,Kladno,720,"164,493",100
4,Central Bohemian Region,Kolín,748,"103,894",90
5,Central Bohemian Region,Kutná Hora,917,"75,683",88
6,Central Bohemian Region,Mělník,701,"109,354",69
7,Central Bohemian Region,Mladá Boleslav,"1,023","127,592",120
8,Central Bohemian Region,Nymburk,846,"101,120",86
9,Central Bohemian Region,Prague-East,755,"188,384",110


In [10]:
regions_data

,Licence\nplate,Region,Capital,Population\n(2022),Area\n(km2),Pop. density\n(/km2),"GDP\n(million CZK, 2022)",GDP per\ncapita
0,A,Prague,Prague,"1,275,406",496,"2,571","1,572,000","1,190,611"
1,S,Central Bohemian,Prague,"1,386,824","10,929",127,"662,000","484,475"
2,C,South Bohemian,České Budějovice,"637,047","10,058",63,"289,000","438,114"
3,P,Plzeň,Plzeň,"578,707","7,649",76,"285,000","474,310"
4,K,Karlovy Vary,Karlovy Vary,"283,210","3,310",86,"97,000","341,512"
5,U,Ústí nad Labem,Ústí nad Labem,"798,898","5,339",150,"313,000","386,363"
6,L,Liberec,Liberec,"437,570","3,163",138,"189,000","411,399"
7,H,Hradec Králové,Hradec Králové,"542,583","4,759",114,"275,000","479,318"
8,E,Pardubice,Pardubice,"514,518","4,519",114,"213,000","422,684"
9,J,Vysočina,Jihlava,"504,025","6,796",74,"228,000","434,018"


The above data was obtained from Wikipedia

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin


class PrahaTransform(BaseEstimator, TransformerMixin):
    
    def check_pattern(self, string):
        parts = string.split('-')
        if len(parts) == 2 and parts[0] == 'praha':
            try:
                num = int(parts[1])
                if 1 <= num <= 99:
                    return True
            except ValueError:
                pass       
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
#         pattern = re.compile('praha-\d{1,2}$')
        X['district'] = X['district'].map(lambda x: 'Praha' if self.check_pattern(x) else x)    
        return X

In [12]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/pythonScripts")
from URL_helper import URL_helper


class AddDistrictsAttributes(BaseEstimator, TransformerMixin):
        
    def fit(self, X, y=None):
        return self
    
    def __extractRegionsData(self, X, regions_data, y=None):
        regions_data = regions_data.drop(['Population\n(2022)', 'Area\n(km2)', 'Pop. density\n(/km2)', 'Licence\nplate'], axis=1)
        regions_data = regions_data.drop(14, axis=0)
        X['Region'] = X['Region'].map(lambda x: x.replace(' Region', ''), na_action='ignore')
        X['District'] = X['District'].map(lambda x: x.replace('-City', ''), na_action='ignore')
        X = pd.merge(regions_data, X)
        X['Is capital'] = np.where(X['Capital'] == X['District'], 1, 0)
        return X
    
    def __extractDistrictsData(self, X, districts_data, y=None):
        helper = URL_helper()
        X['Distric_in_czech'] = X['district'].map(helper.getEnglishNameToCzechDict())
        print(len(districts_data['District'].unique()))
        print(districts_data['District'].unique())
        print('////////')
        print(len(X['Distric_in_czech'].unique()))
        print(X['Distric_in_czech'].unique())
        print(f'In x and not in districts {list(set(X["Distric_in_czech"].unique()) - set(districts_data["District"].unique()))}')
        print(f'In districts and not in X {list(set(districts_data["District"].unique()) - set(X["Distric_in_czech"].unique()))}')
        X = pd.merge(X, districts_data, left_on='Distric_in_czech', right_on='District')
        return X
        
    def transform(self, X, y=None):
        regions_data = pd.read_csv('/Users/eugenganenco/Desktop/srealtyAnalysis/data/Regional Data/Regions_of_the_Czech_Republic_1.csv')
        districts_data = pd.read_csv('/Users/eugenganenco/Desktop/srealtyAnalysis/data/Regional Data/Districts_of_the_Czech_Republic_1.csv')
        
        X = self.__extractDistrictsData(X, districts_data)
        print(f'Shape: {X.shape}')
        X = self.__extractRegionsData(X, regions_data)   
        print(f'Shape: {X.shape}')
        X = X.drop(['district', 'Distric_in_czech', 'Region', 'Capital', 'District', 'Mun.'], axis=1)
        for feature in ['GDP\n(million CZK, 2022)', 'GDP per\ncapita', 'Area\n(km2)', 'Population\n(2022)']:
            if X[feature].dtypes == 'O':               
                X.loc[:,feature] = X.loc[:,feature].map(lambda x: x.replace(',',''), na_action='ignore')
                X.loc[:,feature] = pd.to_numeric(X.loc[:,feature])
        return X

In [13]:
# after AddDistrictsAttributes
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['Population density'] = X['Area\n(km2)'] / X['Population\n(2022)']
        print('Columns:')
        print(X.columns)
        self.columnNames = X.columns
        return X
    
    def get_feature_names(self):
        return self.columnNames

## Data cleaning

### Numercial values functions

In [14]:
class ImputeCoordinates(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X[['locationLat', 'locationLong']] = X[['locationLat', 'locationLong']].apply(pd.to_numeric, errors='coerce')
        X['locationLat'] = X.groupby('district')['locationLat'].transform(lambda x: x.fillna(x.mean()))
        X['locationLong'] = X.groupby('district')['locationLong'].transform(lambda x: x.fillna(x.mean()))
        print(X.shape)
        return X

In [15]:
class ExtractAreaValue(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        featureList = [feature for feature in X.filter(like='area').columns]
        Modified = X.loc[:,featureList].applymap(lambda x: x.replace('m2', ''), na_action='ignore')
        for feature in featureList:
            X[feature] = Modified[feature]
            X[feature] = pd.to_numeric(X[feature])
        return X

In [16]:
#class ExtractYearValue()

In [17]:
# class ExtractRighNumVars(BaseEstimator, TransformerMixin):
#     def fit(self, X, y=None):
#         return self
    
#     def transform(self, X, y=None):
#         X = X.loc[:,['district', 'Usable area', 'Land area', 'proximityIndex', 'Built-up area', 'Garden area', 'Floor area']]
#         return X
    

In [18]:
class DeleteIrrelevantColumns(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = X.loc[:, ~df.columns.str.contains('^Unnamed')]
        X = X.drop(['link', 'location', 'total_price', 'update', 'move_in_date', 'discounted', 'original_price'], inplace=True, axis=1)
        return X

In [19]:
class CustonImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        for feature in [feature for feature in X.columns if X[feature].dtypes != 'O']:
            median=X[feature].median()
            X[feature].fillna(median, inplace=True)
        return X

In [20]:
class Debug(BaseEstimator, TransformerMixin):

    def transform(self, X):
        print('////////////////////////////////////')
        print('////////////////////////////////////')
        print(pd.DataFrame(X).head(5))
        print(X.shape)
        print('////////////////////////////////////')
        print('////////////////////////////////////')
        return X

    def fit(self, X, y=None, **fit_params):
        return self

### Categorical values functions

In [21]:
# class ExtractRightCatVars(BaseEstimator, TransformerMixin):
#     def transform(self, X, y=None):
#         return self
        
#     def fit(self, X, y=None):
#         columnListCategorical = set(X.columns) - set(['Usable area','Land area', 'proximityIndex', 'Built-up area', 'Floor area'])
#         return X[list(columnListCategorical)]

In [22]:
class GroupImputer(BaseEstimator, TransformerMixin):

    def __init__(self, group_cols, Y_train):
        self.group_cols = group_cols
        self.Y_train = Y_train
    
    def fit(self, X, y=None):
        impute_map = X.groupby(self.group_cols)[self.Y_train].agg(pd.Series.mode) \
                                                            .reset_index(drop=False)
        print(impute_map)
        self.impute_map_ = impute_map        
        return self 
    
    def transform(self, X, y=None):
        X = X.copy()       
        for index, row in self.impute_map_.iterrows():
            ind = (X[self.group_cols] == row[self.group_cols]).all(axis=1)
            X.loc[ind, self.Y_train] = X.loc[ind, self.Y_train].fillna(row[self.Y_train])      
        return X.values

### Pipeline construction

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [24]:
#Do something about outliers
from sklearn.preprocessing import FunctionTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

numPipeline = Pipeline([
        ('praha_transform', PrahaTransform()),
#         ('feature_selector', ExtractRighNumVars()),
#        ('ImputeCoordinates', ImputeCoordinates()),
        ('extract_area_vals', ExtractAreaValue()),
        ('add_districs_attributes', AddDistrictsAttributes()),
        ('add_combined_attributes', CombinedAttributesAdder()),
        ('imputer', IterativeImputer(verbose=2, max_iter=30, tol=1e-10)),
        ('LogTransformation', FunctionTransformer(lambda value: np.log1p(value))),
        ('std_scaler', MinMaxScaler())
    ])


In [25]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

catPipeline = Pipeline([
        ('oneHot', OneHotEncoder()),
        ('simpleImputer', SimpleImputer(strategy='most_frequent'))
])

In [26]:
from sklearn.compose import ColumnTransformer

THRESHOLD = 30
numericalValues = ['district', 'usable_area', 'land_area', 'proximityindex', 'built_up_area', 'garden_area', 'floor_area']
irrelevantVariables = ['location', 'auction_decree', 'move_in_date', 'cena', 'index', 'starting_price', 'place_of_auction', 'link',
                       'original_price', 'update', 'sale_start_date', 'locationlong', 'expert_opinion', 'discounted', 
                       'the_date_of_the_auction', 'expert_opinion_1', 'minimum_purchase_price', 'type_of_auction', 'total_price', 'id', 'job_id', 'locationLat']
# eliminate from categorical values variables that are not numeric and the ones that are either irrelevant (link, id) or that are proxy for the actial price ('Original price').
categoricalValues = list(set(X_train.columns) - set(numericalValues) - set(irrelevantVariables))
missingDict = getMissingDict(numericalValues + categoricalValues, X_train)
print(missingDict)
numericalValues = [feature for feature in numericalValues if dict(missingDict)[feature] < THRESHOLD]
categoricalValues = [feature for feature in categoricalValues if dict(missingDict)[feature] < THRESHOLD]
missingDict
print('')
print(f'Categorical  {categoricalValues}')
print('')
print(f'Numerical  {numericalValues}')

columnsPipeline = ColumnTransformer(
         transformers=[
        ('numPipeline', numPipeline, numericalValues),
        ('catPipeline', catPipeline, categoricalValues)
    ]
)

[('the_date_of_the_2nd_inspection', 100.0), ('the_date_of_the_1st_inspection', 100.0), ('minimum_bid', 99.991), ('number_of_owners', 99.991), ('auction_principal', 99.991), ('share_size', 99.973), ('inspection_date_to', 99.919), ('inspection_date', 99.369), ('date_of_completion_of_construction', 99.053), ('pool_area', 98.954), ('number_of_flats', 98.9), ('ceiling_height', 98.873), ('soil_construction', 98.539), ('condition', 98.242), ('housing_costs', 96.627), ('certificate_of_energy_performance_of_the_building', 94.878), ('swimming_pool', 93.065), ('indicator_of_energy_efficiency_of_the_building', 91.821), ('year_of_approval', 91.325), ('year_of_reconstruction', 89.999), ('barrier_free', 79.836), ('telecommunication', 75.237), ('floor_area', 70.863), ('garden_area', 70.665), ('gas', 70.43), ('lift', 67.977), ('garage', 65.885), ('store', 64.532), ('communication', 59.771), ('parking', 56.083), ('note_on_price', 46.082), ('equipment', 40.644), ('transport', 39.192), ('heating', 34.71),

In [27]:
X_train_prepared = columnsPipeline.fit_transform(X_train)

77
['Prague' 'Benešov' 'Beroun' 'Kladno' 'Kolín' 'Kutná Hora' 'Mělník'
 'Mladá Boleslav' 'Nymburk' 'Prague-East' 'Prague-West' 'Příbram'
 'Rakovník' 'České Budějovice' 'Český Krumlov' 'Jindřichův Hradec' 'Písek'
 'Prachatice' 'Strakonice' 'Tábor' 'Domažlice' 'Klatovy' 'Plzeň-City'
 'Plzeň-South' 'Plzeň-North' 'Rokycany' 'Tachov' 'Cheb' 'Karlovy Vary'
 'Sokolov' 'Chomutov' 'Děčín' 'Litoměřice' 'Louny' 'Most' 'Teplice'
 'Ústí nad Labem' 'Česká Lípa' 'Jablonec nad Nisou' 'Liberec' 'Semily'
 'Hradec Králové' 'Jičín' 'Náchod' 'Rychnov nad Kněžnou' 'Trutnov'
 'Chrudim' 'Pardubice' 'Svitavy' 'Ústí nad Orlicí' 'Havlíčkův Brod'
 'Jihlava' 'Pelhřimov' 'Třebíč' 'Žďár nad Sázavou' 'Blansko' 'Břeclav'
 'Brno-City' 'Brno-Country' 'Hodonín' 'Vyškov' 'Znojmo' 'Jeseník'
 'Olomouc' 'Přerov' 'Prostějov' 'Šumperk' 'Kroměříž' 'Uherské Hradiště'
 'Vsetín' 'Zlín' 'Bruntál' 'Frýdek-Místek' 'Karviná' 'Nový Jičín' 'Opava'
 'Ostrava-City']
////////
77
['Náchod' 'Semily' 'Sokolov' 'Prague-West' 'Tábor' 'Litoměřic

In [28]:
# df = pd.DataFrame(_prepared, columns=['GDP\n(million CZK, 2022)', 'GDP per\ncapita', 'Usable area',
#        'Land area', 'proximityIndex', 'Built-up area', 'Floor area',
#        'Area\n(km2)', 'Population\n(2022)', 'Is capital',
#        'Population density'])

In [29]:
# df.head(2000)

In [30]:
X_train_prepared

<11089x10146 sparse matrix of type '<class 'numpy.float64'>'
	with 222295 stored elements in Compressed Sparse Row format>

## Select and train the model

### Training the models

In [31]:
def display_scores(scores, algorithmName):
    print(algorithmName)
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    print()

In [32]:
from sklearn.model_selection import cross_val_score
import numpy as np

# could use dict with algorithm:ParamList as elements of algorithmList or args* for hyperparam tuning.
def runAlgorithmList(algorithmList, X_train, Y_train, scoringFormula="neg_mean_squared_error"):
    algorithmDict = {}
    for algorithm in algorithmList:
        scores = cross_val_score(algorithm, X_train, Y_train,
                         scoring=scoringFormula, cv=10)
        algorithm_scores = np.sqrt(-scores)
        display_scores(algorithm_scores, type(algorithm).__name__)
        algorithmDict[algorithm] = scores.mean()
    return min(algorithmDict, key=algorithmDict.get)

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import dill
import re


bestModel = runAlgorithmList([LinearRegression()], #, SVR(kernel="linear"
                X_train_prepared, Y_train)

finalPipeline = Pipeline([('preprocessor', columnsPipeline),
                         ('model', bestModel)])

finalPipeline.fit(X_train, Y_train)
print(type(finalPipeline))

with open('../telegramBot/model.pickle', 'wb') as f:
    dill.dump(finalPipeline, f)

LinearRegression
Scores: [6534327.58939872 9275998.64502774 7325224.54084781 5927312.99830434
 7364304.50902912 6026906.49092095 6535915.42087906 6673558.16505092
 6707871.82948177 6310226.29619078]
Mean: 6868164.6485131215
Standard deviation: 918821.9405354701

77
['Prague' 'Benešov' 'Beroun' 'Kladno' 'Kolín' 'Kutná Hora' 'Mělník'
 'Mladá Boleslav' 'Nymburk' 'Prague-East' 'Prague-West' 'Příbram'
 'Rakovník' 'České Budějovice' 'Český Krumlov' 'Jindřichův Hradec' 'Písek'
 'Prachatice' 'Strakonice' 'Tábor' 'Domažlice' 'Klatovy' 'Plzeň-City'
 'Plzeň-South' 'Plzeň-North' 'Rokycany' 'Tachov' 'Cheb' 'Karlovy Vary'
 'Sokolov' 'Chomutov' 'Děčín' 'Litoměřice' 'Louny' 'Most' 'Teplice'
 'Ústí nad Labem' 'Česká Lípa' 'Jablonec nad Nisou' 'Liberec' 'Semily'
 'Hradec Králové' 'Jičín' 'Náchod' 'Rychnov nad Kněžnou' 'Trutnov'
 'Chrudim' 'Pardubice' 'Svitavy' 'Ústí nad Orlicí' 'Havlíčkův Brod'
 'Jihlava' 'Pelhřimov' 'Třebíč' 'Žďár nad Sázavou' 'Blansko' 'Břeclav'
 'Brno-City' 'Brno-Country' 'Hodonín' '

### Evaluating the models

## Evaluate Your System on the Test Set

In [34]:
from sklearn.metrics import mean_squared_error
final_model = lin_reg

X_test = test_set.drop('price', axis=1)
y_test = test_set['price'].copy()

X_test_prepared = num_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

NameError: name 'lin_reg' is not defined

In [ ]:
X_test_prepared

In [ ]:
final_model.coef_

'Usable area', 'Land area', 'proximityIndex', 'Built-up area', 'Garden area', 'Floor area'

In [ ]:
final_model.intercept_

In [ ]:
final_rmse

In [ ]:
print(final_predictions[:10])
y_test[:10]